
# EXPLORATORY ESPOL DATA ANALYSIS












## DATA SETS






Primero importamos todas las librerías que vamos a utilizar, posteriormente visualizaciones como esta las datas.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import os
import plotly.express as px
import cufflinks as cf
from IPython.display import display,HTML

cf.set_config_file(sharing="public",theme="ggplot", offline=True)

In [2]:
df_H_p

NameError: name 'df_H_p' is not defined

In [ ]:
df_materias = df_PE_s.drop_duplicates(subset=['NOMBRE'])[['NOMBRE','UNIDAD']]
df_materias['CLASIFICACION'] = ['PAO']*df_materias.shape[0]
df_materias['CLASIFICACION'][np.isin(subjects_all,subjects_pae)]='PAE'
df_materias

In [ ]:
for fac in df_materias.UNIDAD.unique():
    df_materias_fac = df_materias[df_materias.UNIDAD==fac]
    fig = px.treemap(df_materias_fac, path=[px.Constant(f"{fac}"), 'NOMBRE', 'CLASIFICACION'],color='CLASIFICACION',
                color_discrete_map={'PAE':'blue', 'PAO':'red'})
    fig.show()

In [ ]:
df_H_p.tail()


In [ ]:
def convert_time(df,feature):
    li = []
    for x in list(df[feature]):
        li_ho = x.split(' ')
        if 'a' in li_ho[1]:
            li_t = li_ho[0].split(':')
            if len(li_t[0]) == 1:
                n = '0'+li_ho[0] + 'AM'
                li.append(n)
            elif len(li_t[0]) > 1:
                n = li_ho[0] + 'AM'
                li.append(n)

        elif 'p' in li_ho[1]:
            li_t = li_ho[0].split(':')
            if 1 == len(li_t[0]):
                n = '0' + li_ho[0] + 'PM'
                li.append(n)
            elif len(li_t[0]) > 1:
                n = li_ho[0] + 'PM'
                li.append(n)

    df[feature] = li

In [ ]:
df_H_p

In [ ]:
import datetime as dt
df_H_p['HORAINICIO'] = pd.to_datetime(df_H_p['HORAINICIO'],format="%I:%M:%S%p")
df_H_p['HORAFIN'] = pd.to_datetime(df_H_p['HORAFIN'],format="%I:%M:%S%p")
df_H_p['DURACION'] = (df_H_p['HORAFIN']-df_H_p['HORAINICIO'])
df_H_p['HORAINICIO'] = [dt.datetime.strptime(str(t),'%Y-%m-%d %H:%M:%S').time() for  t in df_H_p['HORAINICIO']]
df_H_p['HORAFIN'] = [dt.datetime.strptime(str(t),'%Y-%m-%d %H:%M:%S').time() for  t in df_H_p['HORAFIN']]
df_H_p['DURACION'] = [(str(d).split('s')[1].strip()) for d in df_H_p['DURACION']]

In [ ]:
def convert_int(df,name_column):
    df[name_column] = [(n.split(",")[0])for n in df[name_column]]
    df[name_column] = df_H_p[name_column].astype(int)
    return df

for column in ['PARALELO','CAPACIDAD','CUPOPLANIFICADO']:
    convert_int(df_H_p,column)

In [ ]:
df_H_p.info()

In [ ]:
df_H_p['ESQUINCENA'].value_counts()

In [ ]:
df_H_p['UNIDAD'].value_counts()

In [ ]:
df_H_p[['AULA', 'UNIDAD']].value_counts()

## HORARIO POR MATERIA


Podemos armar un horario de la hora de entra y salida por materias.

Primero podemos observar que tenemos 17 materias en el PAE

In [ ]:
df_H_p.NOMBRE.nunique()

In [ ]:
df_H_p.query("NOMBRE == 'TÉCNICAS DIETÉTICAS'")


In [ ]:
cond=[df_H_p['PARALELO'] ==100,df_H_p['PARALELO'] ==101,df_H_p['PARALELO']==102,df_H_p['PARALELO']==103,df_H_p['PARALELO']==104,df_H_p['PARALELO']==105,df_H_p['PARALELO']==106]
result=[1,1,2,3,4,5,6]
df_H_p['COURSE']=np.select(cond,result,default=df_H_p['PARALELO'])
#solo tiene un paralelo
df_H_p['COURSE'][np.isin(df_H_p['NOMBRE'],['SISTEMAS DIGITALES I','PRINCIPIOS DE ELECTRÓNICA'])] = 1
cond=[df_H_p['PARALELO']>=100,df_H_p['PARALELO']<100]
result=['Práctico','Teórico']
df_H_p['CLASE']=np.select(cond,result)

La materia de TÉCNICAS DIETÉTICAS tiene un solo paparelo de teórico, pero dos de practico donde la mitad de los alumnos va a uno y la otra mitad al otro, fuente:  https://www.academico.espol.edu.ec/UI/Registros/MateriaPlanificada.aspx?idMateria=6973&numeroParalelo=1&p=587&idCurso=159345 ,  no obstante, se puede ver el caso de dos materias donde existe un solo curso teórico, pero este  a su vez se divide en varios prácticos.

Principios de electrónica.

Sistemas Digitales.

In [ ]:
X = df_H_p[[ 'DIA','HORAINICIO','HORAFIN','PARALELO','NOMBRE','DOCENTE','COURSE','CLASE','CAPACIDAD','CUPOPLANIFICADO']][np.isin(df_H_p['NOMBRE'],['SISTEMAS DIGITALES I','PRINCIPIOS DE ELECTRÓNICA'])].groupby(['NOMBRE','DOCENTE','PARALELO','HORAINICIO','CAPACIDAD','CUPOPLANIFICADO',
                                                                                                                                                                                     'HORAFIN','DIA','CLASE']).count()
X
#Como podemos ver de un paralelo teórico , dividieron en 3 paralelos prácticos.

In [ ]:
df_H_p

Existen casos donde hay varios cursos para la misma materia

In [ ]:
X = df_H_p[[ 'DIA','HORAINICIO','HORAFIN','PARALELO','NOMBRE','DOCENTE','COURSE','CLASE','CAPACIDAD','CUPOPLANIFICADO']][np.isin(df_H_p['NOMBRE'],['INGLÉS V'])].groupby(['NOMBRE','DOCENTE','PARALELO','HORAINICIO','CAPACIDAD','CUPOPLANIFICADO',
                                                                                                                                                                                     'HORAFIN','DIA','CLASE']).count()
X

In [ ]:
df_H_p['NOMBRE'].value_counts()

Crear data para gráfica

In [ ]:
df_Horario = pd.DataFrame(columns=['NOMBRE', 'HORARIO','INDICAR','PARALELO','DIA','COURSE','CLASE'])
df_Horario['NOMBRE']=list(df_H_p['NOMBRE'])*2
df_Horario['HORARIO']=list(df_H_p['HORAINICIO'])+list(df_H_p['HORAFIN'])
df_Horario['INDICAR'] = (['ENTRADA']*127)+(["SALIDA"]*127)
df_Horario['PARALELO'] = list(df_H_p['PARALELO'])*2
df_Horario['DIA']=list(df_H_p['DIA'])*2
df_Horario['COURSE'] = list(df_H_p['COURSE'])*2
df_Horario['CLASE'] = list(df_H_p['CLASE'])*2
dias_ordenados = ["lunes", "martes", "miércoles", "jueves"]
df_Horario.DIA = pd.Categorical(df_Horario['DIA'], dias_ordenados)
df_Horario = df_Horario.sort_values(by=['DIA','COURSE'])
df_Horario['size']=[10]*254
df_Horario

In [ ]:
U = df_Horario.query("NOMBRE=='INGLÉS V'")
U[U['COURSE']==1]

In [ ]:
df_Horario[df_Horario['COURSE']==1]

In [ ]:
import plotly.express as px
materias = list(df_H_p.NOMBRE.unique())
for mate in materias:
    df = df_Horario[df_Horario.NOMBRE == mate]
    l_course = df.COURSE.unique()
    for n_course in l_course:
        df_course = df[df.COURSE == n_course]
        fig = px.scatter(df_course, y="DIA", x="HORARIO",color='INDICAR',symbol='INDICAR',size='size',hover_data=['PARALELO','CLASE'],title=f'{mate} Course {n_course}')
        fig.show()


Podria borrar aqui

In [ ]:
df_Horario.DIA.value_counts()

In [ ]:
U = df_Horario.query("NOMBRE=='INGLÉS V'")
U

In [ ]:
U = U.drop('PARALELO',axis=1)
t =  U.groupby(['COURSE']).get_group(1).value_counts()
t


In [ ]:
list(df_Horario['DIA'].unique())

## MAPA

In [ ]:
pd.set_option('display.max_columns', 500)
#una materia 

In [ ]:
pd.set_option('display.max_rows', 50)

In [ ]:
dic_ubi={
    '9M':[-2.14662, -79.96666],
    '3H':[-2.1523, -79.95582],
    'GEA':[-2.14016, -79.96282],
    '13B':[-2.14558, -79.96507],
    '7B':[-2.14704, -79.96609] ,
    '11A':[-2.14466, -79.96772],
    '8F':[-2.14878, -79.96752],
    '8E':[-2.14873, -79.96782],
    '9F':[-2.14715, -79.96778],
    '11C':[-2.14563, -79.96683],
    '11D':[-2.14544, -79.96601],
    '3K':[-2.15122, -79.95424],
    
}

In [ ]:
df_H_p['BLOQUE']=[(i.strip())for i in df_H_p['BLOQUE']]
df_H_p['lat']=[(dic_ubi[bl][0])for bl in list(df_H_p['BLOQUE'])]
df_H_p['lon']=[(dic_ubi[bl][1])for bl in list(df_H_p['BLOQUE'])]

In [ ]:
espol_ubi = df_H_p[['NOMBRE','BLOQUE','CAPACIDAD','CLASE','lat','lon']].groupby(['NOMBRE', 'BLOQUE','CAPACIDAD','CLASE','lat','lon'], as_index= False).count()

In [ ]:
def normalizar_lon(df,bloque,lon):
    modiCor=[]
    compa=[]
    cont=0
    for bloque,lo in zip (df[bloque],df[lon]):
        if bloque not in compa:
            compa.append(bloque)
            modiCor.append(lo)
        else:
            cont=cont+0.000009
            
            modiCor.append(lo-cont)
    
    df[lon+'2']=modiCor

In [ ]:
def normalizar_lat(df,bloque,lat):
    modiCor=[]
    compa=[]
    cont=0
    for bloque,lo in zip (df[bloque],df[lat]):
        if bloque not in compa:
            compa.append(bloque)
            modiCor.append(lo)
        else:
            cont=cont+0.0000009
            
            modiCor.append(lo-cont)
    
    df[lat+'2']=modiCor

In [ ]:
normalizar_lon(espol_ubi,'BLOQUE','lon')
normalizar_lat(espol_ubi,'BLOQUE','lat')
espol_ubi

In [ ]:
#px.set_mapbox_access_token('pk.eyJ1Ijoiam9udGh6IiwiYSI6ImNsZ3VkNnB4ZDJicW8zdGx1cXR5c21mcjAifQ.80mZH8cBu3LlgBN_0s8BKQ')
#se opto por usar el open street map porque esta mas bonito 

fig1 = px.scatter_mapbox(espol_ubi, lat="lat2", lon="lon2", hover_data=["BLOQUE", "CAPACIDAD", 'CLASE'],
                         zoom=15, height=600,color_continuous_scale=px.colors.cyclical.IceFire,color='NOMBRE',size='CAPACIDAD')
fig1.update_layout(mapbox_style="open-street-map", mapbox=dict(
        center=dict(
            lat=-2.14756,
            lon=-79.9645
        ),))
fig1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig1.show()

## Cantidad de paralelos 

In [ ]:
b =df_H_p[["PARALELO",'CLASE','UNIDAD', 'BLOQUE','NOMBRE']].groupby(["PARALELO",'CLASE', 'UNIDAD', 'BLOQUE','NOMBRE'], as_index= False).count()
b

In [ ]:
s=pd.DataFrame(b[['NOMBRE']].value_counts()) 
s=s.reset_index()
s.columns=['Materias' ,'CantParalelos']
s

In [ ]:
fig = px.line(s, x='Materias', y="CantParalelos",markers = True,text="CantParalelos",title='Cantidad')
fig.update_traces(textposition = "top center")
fig.show()  

## Capacidad por hora

In [ ]:
df_H_p

In [ ]:
df_Horario

In [ ]:
HI = df_H_p.groupby('HORAINICIO',as_index=False)['CAPACIDAD'].sum()
HF = df_H_p.groupby('HORAFIN',as_index=False)['CAPACIDAD'].sum()
HI,HF

In [ ]:
h_inicial= dt.datetime(2023, 1, 1, 9, 0, 0)
h_final=dt.datetime(2023, 1, 1, 17, 1, 0)
hora=dt.timedelta(hours=1)
time_1Hour =[(str(x).split('T')[1]).split('.')[0] for x in np.arange(h_inicial,h_final,hora)]
time_1Hour

In [ ]:
def capacidad_porhora(df,name_column,name_capacidad):
    i=0
    historia=0
    for hi in df[name_column]:
        valida= str(hi).split(':')[1]
        if valida == '00':
            df.loc[i,name_capacidad]+=historia
            historia=0
        else:
            historia = df.loc[i,name_capacidad]
            df.drop(i,axis=0,inplace=True)
        i+=1
    return df
HI,HF = capacidad_porhora(HI,'HORAINICIO','CAPACIDAD').reset_index(drop=True),capacidad_porhora(HF,'HORAFIN','CAPACIDAD').reset_index(drop=True)
HI.HORAINICIO=HI.HORAINICIO.astype(str)
HF.HORAFIN=HF.HORAFIN.astype(str)

In [ ]:
for hora in time_1Hour:
    if hora not in list(HI['HORAINICIO']):
        HI.loc[len(HI)]=[hora,0]
    if hora not in list(HF['HORAFIN']):
        HF.loc[len(HF)]=[hora,0]



In [ ]:
HI.HORAINICIO= pd.Categorical(HI.HORAINICIO,time_1Hour)
HI.HORAFIN=pd.Categorical(HF.HORAFIN,time_1Hour)
HI=HI.sort_values('HORAINICIO').reset_index(drop=True)
HF=HF.sort_values('HORAFIN').reset_index(drop=True)


In [ ]:
df_capacidad = pd.DataFrame(index=time_1Hour,columns=['ENTRADA','SALIDA'])
df_capacidad['ENTRADA']=HI['CAPACIDAD'].values
df_capacidad['SALIDA']=HF['CAPACIDAD'].values

In [ ]:
columns = list(df_capacidad.columns)

In [ ]:
import plotly.express as px
fig10 = px.line(df_capacidad, x=df_capacidad.index, y=columns , title='FLUJOS DE ESTUDIANTES ', labels={'index':'HORARIOS','value':'NUMERO DE ESTUDIANTES'})
fig10.update_traces(textposition = "top center")
fig10.update_layout( # customize font and legend orientation & position
    font_family="Rockwell",
    legend=dict(
        title='TIEMPOS', orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ))
fig10.show()

In [ ]:
df7 = df_H_p.groupby(["UNIDAD","DIA","PARALELO","HORAINICIO","HORAFIN"],as_index=False)['CUPOPLANIFICADO'].sum()
df7.value_counts()

In [ ]:
df_H_p.query("UNIDAD == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL' and DIA =='lunes' ")

In [ ]:
df_H_p.groupby(['UNIDAD'],as_index=False)[['HORAINICIO','HORAFIN','CUPOPLANIFICADO']].value_counts()

## Duracion


In [ ]:
dura =df_H_p[["PARALELO", 'CUPOPLANIFICADO' ,'CLASE','UNIDAD','BLOQUE' ,'NOMBRE', 'DIA','DURACION']].groupby(["PARALELO",'CUPOPLANIFICADO' ,'CLASE','UNIDAD','BLOQUE' ,'NOMBRE', 'DIA','DURACION'], as_index= False).count()
dura

In [ ]:
dura = dura.sort_values('DURACION')

In [ ]:
dura

In [ ]:
unidades = list(dura.UNIDAD.unique())
for unid in unidades:
    df_t = dura[dura.UNIDAD == unid]
    df_t
    if list(df_t['UNIDAD'])[0] == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL':
        df_t.loc['200'] = [1, 'Teórico', '0','ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']
        fig_dura = px.bar(df_t, x="DURACION", y="NOMBRE", color="NOMBRE", title=unid, height=800, labels={'NOMBRE': 'MATERIAS'}, facet_col='DIA',  facet_col_wrap=1, hover_data=['DURACION', 'PARALELO', 'CLASE','CUPOPLANIFICADO'], barmode='group')
        fig_dura.update_layout(
            font_family="Rockwell"
        )
        fig_dura.update_yaxes(
          showgrid=False)
        fig_dura.show()
    else:
        df_t.loc['200'] = [1, 'Teórico', '0','ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']
        fig_dura = px.bar(df_t, x="DURACION", y="NOMBRE", color="NOMBRE", title=unid, height=800, facet_col='DIA', labels={'NOMBRE': 'MATERIAS'}, facet_col_wrap=1, hover_data=['DURACION', 'PARALELO', 'CLASE', 'CUPOPLANIFICADO'], barmode='group')
        fig_dura.update_layout(
            font_family="Rockwell"
        )
        fig_dura.update_yaxes(
          showgrid=False)
        fig_dura.show()
   

In [ ]:
dura.query("NOMBRE == 'EMPRENDIMIENTO E INNOVACIÓN'")

In [ ]:
'''df_t1 = dura[dura.UNIDAD == 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL']
df_t1.loc['200'] = [1, 'Teórico', 'ESCUELA SUPERIOR POLITECNICA DEL LITORAL', '7B', 'MATERIA', 'lunes', '03:00:00']
df_t1'''

## IRREGULARIDADES


In [ ]:
df_H_p.query("NOMBRE == 'INGLÉS I'")
## No exite paralelo 2 a diferencia de lo que esta en el academico

In [ ]:
df_H_p.NOMBRE.unique()

In [ ]:
#Los cupos en emprendimiento e innovación tienen inconsistencias con respecto al data set entregado

## FLEUNCIA ESTIMADA DE ESTUDIANTES RECIBIENDO CLASES POR FACULTAD

Nos permite ver como es el flujo de estudiantes que están recibiendo clases por facultad y como esto va cambiando durante el dia.
Para este proceso primero eliminamos las filas repetidas . 


In [ ]:
df_H_p.drop_duplicates()